In [ ]:
import time
from scipy import sparse
from math import ceil
import pandas as pd
import numba as nb
import sys
eps =1  # 与SPONGE单位制匹配  #0.08854187817 # F/A
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from scipy.linalg import lu,solve
from mpl_toolkits.mplot3d import Axes3D
from skimage import measure


In [ ]:
def waiting(i,N):
    print("\r", end="")
    print("进度: {}%: ".format(ceil(i/N*100)), "▓" * (ceil(i/N*100) // 2), end="")
    sys.stdout.flush()
    time.sleep(0.01)

网格划分

In [ ]:
box_size = np.array([100.0,100.0,100.0]).astype("float64")
(NX,NY,NZ) = (100,100,100)
(dx,dy,dz) = (box_size[0]/NX, box_size[1]/NY, box_size[2]/NZ)

# 在格子中内画个球, 球心(15.0, 15.0, 15.0), 半径 5.5
# 设置等势值
iso_level = 1
def sphere(crd):
     if crd[0] == 50.0 and crd[1] == 50.0 and crd[2] == 50.0 :
          return 1e10
     else:
          return 20.38/np.sqrt((crd[0]-50.0)**2 + (crd[1]-50.0)**2 + (crd[2]-50.0)**2)
field = np.zeros((NX+1,NY+1,NZ+1))
for i in range(NX+1):
     for j in range(NY+1):
          for k in range(NZ+1):
               field[i,j,k] = sphere(np.array([i*dx,j*dy,k*dz]))
               
# 给出切点的坐标，以及几何序号
cut_crd = measure.marching_cubes(field, iso_level,spacing=(dx, dy, dz))[0]
cut_cnt = cut_crd.shape[0]
cut_sgn = [int(i+(NX)*(NY)*(NZ)) for i in range(cut_cnt)]

# 给出需要计算的节点的全局序号
sgn_convert_to_calc = {}
# 全局序号的个数
sgn_calc_cnt = 0

now_sgn = 0
for k in range(NZ):
     for j in range(NY):
          for i in range(NX):
               sgn = i + j*NX + k*NX*NY
               if field[i,j,k] < iso_level:
                    sgn_convert_to_calc[sgn] = now_sgn
                    now_sgn += 1
for i in range(cut_cnt):
     sgn_convert_to_calc[cut_sgn[i]] = now_sgn
     now_sgn += 1
     
calc_convert_to_sgn = {v: k for k, v in sgn_convert_to_calc.items()}
sgn_calc_cnt = len(sgn_convert_to_calc)
first_cut_sgn = sgn_calc_cnt-cut_cnt

## 映射计算
calc_need_convert_list = []
for k in range(NZ):
     for j in range(NY):
          for i in range(NX):
               sgn = i + j*NX + k*NX*NY
               if field[i,j,k] < iso_level:
                    calc_need_convert_list.append(int(sgn))

## 映射计算
now_sgn = 0
ortho_need_convert_list = []
for k in range(NZ):
     for j in range(NY):
          for i in range(NX):
               if field[i,j,k] < iso_level:
                    ortho_need_convert_list.append(now_sgn)
                    now_sgn += 1
               else:
                    ortho_need_convert_list.append(-1)


规整网格 -> 不规则网格插值 & inverse \
定义cubes类与坐标转换，定义四面体在cubes内部存储

In [ ]:
# 注意：Cube
class Cube:
    def __init__(self,i,j,k): # 0 <= i,j,k <= 99 
        self.sgn = i + j*NX + k*NX*NY
        self.crd = np.array([[i*dx,j*dy,k*dz],
                             [(i+1)*dx,j*dy,k*dz],
                             [(i+1)*dx,(j+1)*dy,k*dz],
                             [i*dx,(j+1)*dy,k*dz],
                             [i*dx,j*dy,(k+1)*dz],
                             [(i+1)*dx,j*dy,(k+1)*dz],
                             [(i+1)*dx,(j+1)*dy,(k+1)*dz],
                             [i*dx,(j+1)*dy,(k+1)*dz]])
        self.points_sgn = np.array([i + j*NX + k*NX*NY,
                              (i+1)%NX + j*NX + k*NX*NY,
                              (i+1)%NX + (j+1)%NY*NX + k*NX*NY,
                              i + (j+1)%NY*NX + k*NX*NY,
                              i + j*NX + (k+1)%NZ*NX*NY,
                              (i+1)%NX + j*NX + (k+1)%NZ*NX*NY,
                              (i+1)%NX + (j+1)%NY*NX + (k+1)%NZ*NX*NY,
                              i + (j+1)%NY*NX + (k+1)%NZ*NX*NY])
        self.cut_crd = []
        self.cut_sgn = []
        self.tetra = None
        # 划定顶点位置，判断是否需要计算，判断是否按照四面体元计算
        self.if_out = np.zeros(8)
        self.if_calc = True
        self.if_tetra = False
        cnt = 0
        for i in range(8):
            if sphere(self.crd[i]) < iso_level:
                self.if_out[i] = 1
            else:
                self.if_out[i] = 0
                cnt += 1
        if cnt == 8:
            self.if_calc = False
        if cnt >0:
            self.if_tetra = True
            
        # 引入切点,输入的是几何序号！
    def add_point(self,crd,sgn):
        self.cut_crd.append(crd)
        self.cut_sgn.append(sgn)
        
    def delaunay(self):
        if self.if_calc != True or self.if_tetra != True:
            return
        else:
            self.tetra_crd = []
            self.tetra_sgn = []
            for i in range(8):
                if self.if_out[i] ==1:
                    self.tetra_crd.append(self.crd[i])
                    self.tetra_sgn.append(self.points_sgn[i])
            for i in range(len(self.cut_sgn)):
                self.tetra_crd.append(self.cut_crd[i])
                self.tetra_sgn.append(self.cut_sgn[i])
            #print(self.tetra_crd)
            self.tetra = Delaunay(self.tetra_crd).simplices
            self.tetra_crd = np.array(self.tetra_crd)
            self.tetra_sgn = np.array(self.tetra_sgn)
            return
            
def ijk_to_sgn(i,j,k):
    (l,m,n) = (i%NX,j%NY,k%NZ)
    return int(l + m*NX + n*NX*NY)

# 初始化网格
cubes = []
for k in range(NZ):
    ### 进度条
    waiting(k,NZ)
    for j in range(NY):
        for i in range(NX):
            cubes.append(Cube(i,j,k))

In [ ]:
# 加入节点
def add_cut_in_cubes(crd,sgn):
    (i,j,k) = (crd[0]//dx, crd[1]//dy, crd[2]//dz)
    (if_x, if_y, if_z) = (crd[0]%dx, crd[1]%dy, crd[2]%dz)
    
    if if_x != 0:
        cubes[ijk_to_sgn(i,j,k)].add_point(crd,sgn)
        cubes[ijk_to_sgn(i,j-1,k)].add_point(crd,sgn)
        cubes[ijk_to_sgn(i,j-1,k-1)].add_point(crd,sgn)
        cubes[ijk_to_sgn(i,j,k-1)].add_point(crd,sgn)
    elif if_y != 0:
        cubes[ijk_to_sgn(i,j,k)].add_point(crd,sgn)
        cubes[ijk_to_sgn(i-1,j,k)].add_point(crd,sgn)
        cubes[ijk_to_sgn(i-1,j,k-1)].add_point(crd,sgn)
        cubes[ijk_to_sgn(i,j,k-1)].add_point(crd,sgn)
    elif if_z != 0:
        cubes[ijk_to_sgn(i,j,k)].add_point(crd,sgn)
        cubes[ijk_to_sgn(i,j-1,k)].add_point(crd,sgn)
        cubes[ijk_to_sgn(i-1,j-1,k)].add_point(crd,sgn)
        cubes[ijk_to_sgn(i-1,j,k)].add_point(crd,sgn)
        
        
for i in range(cut_cnt):
    add_cut_in_cubes(cut_crd[i],cut_sgn[i])
    
# 生成四面体
# 初始化网格
for cube in cubes:
    cube.delaunay()
    
# 生成电势项求解矩阵
Phi = np.zeros(first_cut_sgn)
iso_surf_phi = 1
Solvation_Const = np.zeros(first_cut_sgn)
Solvation_Matrix = sparse.lil_matrix((first_cut_sgn,first_cut_sgn),dtype="float64")

In [ ]:
# generate Matrix from cubes which contains tetrahedron
def from_tetra_generate_deviation(i,j,k):
    cube = cubes[ijk_to_sgn(i,j,k)]
    tetras =  cube.tetra
    for tet in tetras:
        # 初始化四面体格点坐标crd[i]、电势序号sgn[i]
        crd = np.zeros((4,3))
        sgn = np.zeros(4).astype(int)
        for q in range(4):
            crd[q] = cube.tetra_crd[tet[q]]
            sgn[q] = int(sgn_convert_to_calc[cube.tetra_sgn[tet[q]]])
        
        # 初始化系数
        Ve = np.abs((1/6)*np.linalg.det([[1,crd[0,0],crd[0,1],crd[0,2]],
                                  [1,crd[1,0],crd[1,1],crd[1,2]],
                                  [1,crd[2,0],crd[2,1],crd[2,2]],
                                  [1,crd[3,0],crd[3,1],crd[3,2]]]))
        # DEBUG 4-24
        if Ve == 0:
            print("error! Ve = 0")
            print(i,j,k)
            print(crd)
            continue
        cst = eps/(72*Ve)
        
        a1 = -np.linalg.det([[1,crd[1,1],crd[1,2]], [1,crd[2,1],crd[2,2]], [1,crd[3,1],crd[3,2]]])
        a2 = np.linalg.det([[1,crd[0,1],crd[0,2]], [1,crd[2,1],crd[2,2]], [1,crd[3,1],crd[3,2]]])
        a3 = -np.linalg.det([[1,crd[0,1],crd[0,2]], [1,crd[1,1],crd[1,2]], [1,crd[3,1],crd[3,2]]])
        
        b1 = np.linalg.det([[1,crd[1,0],crd[1,2]], [1,crd[2,0],crd[2,2]], [1,crd[3,0],crd[3,2]]])
        b2 = -np.linalg.det([[1,crd[0,0],crd[0,2]], [1,crd[2,0],crd[2,2]], [1,crd[3,0],crd[3,2]]])
        b3 = np.linalg.det([[1,crd[0,0],crd[0,2]], [1,crd[1,0],crd[1,2]], [1,crd[3,0],crd[3,2]]])

        c1 = -np.linalg.det([[1,crd[1,0],crd[1,1]], [1,crd[2,0],crd[2,1]], [1,crd[3,0],crd[3,1]]])
        c2 = np.linalg.det([[1,crd[0,0],crd[0,1]], [1,crd[2,0],crd[2,1]], [1,crd[3,0],crd[3,1]]])
        c3 = -np.linalg.det([[1,crd[0,0],crd[0,1]], [1,crd[1,0],crd[1,1]], [1,crd[3,0],crd[3,1]]])
        
        if sgn[0] < first_cut_sgn: # 如果sgn[0] 不是切点，正常计算
            Solvation_Matrix[sgn[0],sgn[0]] += cst*(2*a1*a1+2*b1*b1+2*c1*c1)
            if sgn[1] <first_cut_sgn:
                Solvation_Matrix[sgn[0],sgn[1]] += cst*(2*a1*a2+2*b1*b2+2*c1*c2)
            else:
                Solvation_Const[sgn[0]] -= cst*(2*a1*a2+2*b1*b2+2*c1*c2)*iso_surf_phi
                
            if sgn[2] <first_cut_sgn:
                Solvation_Matrix[sgn[0],sgn[2]] += cst*(2*a1*a3+2*b1*b3+2*c1*c3)
            else:
                Solvation_Const[sgn[0]] -= cst*(2*a1*a3+2*b1*b3+2*c1*c3)*iso_surf_phi
                
            if sgn[3] <first_cut_sgn:
                Solvation_Matrix[sgn[0],sgn[3]] += cst*(-2*a1*(a1+a2+a3)-2*b1*(b1+b2+b3)-2*c1*(c1+c2+c3))
            else:
                Solvation_Const[sgn[0]] -= cst*(-2*a1*(a1+a2+a3)-2*b1*(b1+b2+b3)-2*c1*(c1+c2+c3))*iso_surf_phi
                
        if sgn[1] < first_cut_sgn:
            Solvation_Matrix[sgn[1],sgn[1]] += cst*(2*a2*a2+2*b2*b2+2*c2*c2)
            if sgn[0] <first_cut_sgn:
                Solvation_Matrix[sgn[1],sgn[0]] += cst*(2*a2*a1+2*b2*b1+2*c2*c1)
            else:
                Solvation_Const[sgn[1]] -= cst*(2*a2*a1+2*b2*b1+2*c2*c1)*iso_surf_phi
            
            if sgn[2] <first_cut_sgn:
                Solvation_Matrix[sgn[1],sgn[2]] += cst*(2*a2*a3+2*b2*b3+2*c2*c3)
            else:
                Solvation_Const[sgn[1]] -= cst*(2*a2*a3+2*b2*b3+2*c2*c3)*iso_surf_phi
            
            if sgn[3] <first_cut_sgn:
                Solvation_Matrix[sgn[1],sgn[3]] += cst*(-2*a2*(a1+a2+a3)-2*b2*(b1+b2+b3)-2*c2*(c1+c2+c3))
            else:
                Solvation_Const[sgn[1]] -= cst*(-2*a2*(a1+a2+a3)-2*b2*(b1+b2+b3)-2*c2*(c1+c2+c3))*iso_surf_phi
        
        if sgn[2] < first_cut_sgn:  
            Solvation_Matrix[sgn[2],sgn[2]] += cst*(2*a3*a3+2*b3*b3+2*c3*c3)
            if sgn[0] <first_cut_sgn:
                Solvation_Matrix[sgn[2],sgn[0]] += cst*(2*a3*a1+2*b3*b1+2*c3*c1)
            else:
                Solvation_Const[sgn[2]] -= cst*(2*a3*a1+2*b3*b1+2*c3*c1)*iso_surf_phi
            
            if sgn[1] <first_cut_sgn:
                Solvation_Matrix[sgn[2],sgn[1]] += cst*(2*a3*a2+2*b3*b2+2*c3*c2)
            else:
                Solvation_Const[sgn[2]] -= cst*(2*a3*a2+2*b3*b2+2*c3*c2)*iso_surf_phi
            
            if sgn[3] < first_cut_sgn:  
                Solvation_Matrix[sgn[2],sgn[3]] += cst*(-2*a3*(a1+a2+a3)-2*b3*(b1+b2+b3)-2*c3*(c1+c2+c3))
            else:
                Solvation_Const[sgn[2]] -= cst*(-2*a3*(a1+a2+a3)-2*b3*(b1+b2+b3)-2*c3*(c1+c2+c3))*iso_surf_phi
        
        
        if sgn[3] < first_cut_sgn:    
            Solvation_Matrix[sgn[3],sgn[3]] += cst*(2*(a1+a2+a3)**2+2*(b1+b2+b3)**2+2*(c1+c2+c3)**2)
            if sgn[0] <first_cut_sgn:
                Solvation_Matrix[sgn[3],sgn[0]] += cst*(-2*a1*(a1+a2+a3)-2*b1*(b1+b2+b3)-2*c1*(c1+c2+c3))
            else:
                Solvation_Const[sgn[3]] -= cst*(-2*a1*(a1+a2+a3)-2*b1*(b1+b2+b3)-2*c1*(c1+c2+c3))*iso_surf_phi
            
            if sgn[1] <first_cut_sgn:
                Solvation_Matrix[sgn[3],sgn[1]] += cst*(-2*a2*(a1+a2+a3)-2*b2*(b1+b2+b3)-2*c2*(c1+c2+c3))
            else:
                Solvation_Const[sgn[3]] -= cst*(-2*a2*(a1+a2+a3)-2*b2*(b1+b2+b3)-2*c2*(c1+c2+c3))*iso_surf_phi
                
            if sgn[2] <first_cut_sgn:    
                Solvation_Matrix[sgn[3],sgn[2]] += cst*(-2*a3*(a1+a2+a3)-2*b3*(b1+b2+b3)-2*c3*(c1+c2+c3))
            else:
                Solvation_Const[sgn[3]] -= cst*(-2*a3*(a1+a2+a3)-2*b3*(b1+b2+b3)-2*c3*(c1+c2+c3))*iso_surf_phi
           
# generate Matrix from pure cubes
# 从一个Cube中generate 偏微分贡献，加入Matrix中
# Debug 4-16 没有+= 
# DEBUG 检查对称性，从而判断序号是否正确

def from_cube_generate_deviation(i,j,k):
    phi_sgn = [sgn_convert_to_calc[i] for i in cubes[ijk_to_sgn(i,j,k)].points_sgn]
    (a,b,c) = (eps*dx*dy/(dz*9), eps*dx*dz/(dy*9), eps*dy*dz/(dx*9))
    Solvation_Matrix[phi_sgn[0],phi_sgn[0]] += a + b + c
    Solvation_Matrix[phi_sgn[0],phi_sgn[1]] += 0.5*a + 0.5*b - c
    Solvation_Matrix[phi_sgn[0],phi_sgn[2]] += 0.25*a - 0.5*b - 0.5*c
    Solvation_Matrix[phi_sgn[0],phi_sgn[3]] += 0.5*a - b + 0.5*c
    Solvation_Matrix[phi_sgn[0],phi_sgn[4]] += -a + 0.5*b + 0.5*c
    Solvation_Matrix[phi_sgn[0],phi_sgn[5]] += -0.5*a + 0.25*b - 0.5*c
    Solvation_Matrix[phi_sgn[0],phi_sgn[6]] += -0.25*a - 0.25*b - 0.25*c
    Solvation_Matrix[phi_sgn[0],phi_sgn[7]] += -0.5*a - 0.5*b + 0.25*c
    
    Solvation_Matrix[phi_sgn[1],phi_sgn[0]] += 0.5*a + 0.5*b - c
    Solvation_Matrix[phi_sgn[1],phi_sgn[1]] += a + b + c
    Solvation_Matrix[phi_sgn[1],phi_sgn[2]] += 0.5*a - b + 0.5*c
    Solvation_Matrix[phi_sgn[1],phi_sgn[3]] += 0.25*a - 0.5*b - 0.5*c
    Solvation_Matrix[phi_sgn[1],phi_sgn[4]] += -0.5*a + 0.25*b - 0.5*c
    Solvation_Matrix[phi_sgn[1],phi_sgn[5]] += -a + 0.5*b + 0.5*c
    Solvation_Matrix[phi_sgn[1],phi_sgn[6]] += -0.5*a - 0.5*b + 0.25*c
    Solvation_Matrix[phi_sgn[1],phi_sgn[7]] += -0.25*a - 0.25*b - 0.25*c
    
    Solvation_Matrix[phi_sgn[2],phi_sgn[0]] += 0.25*a - 0.5*b - 0.5*c
    Solvation_Matrix[phi_sgn[2],phi_sgn[1]] += 0.5*a - b + 0.5*c
    Solvation_Matrix[phi_sgn[2],phi_sgn[2]] += a + b + c
    Solvation_Matrix[phi_sgn[2],phi_sgn[3]] += 0.5*a + 0.5*b - c
    Solvation_Matrix[phi_sgn[2],phi_sgn[4]] += -0.25*a - 0.25*b - 0.25*c
    Solvation_Matrix[phi_sgn[2],phi_sgn[5]] += -0.5*a - 0.5*b + 0.25*c
    Solvation_Matrix[phi_sgn[2],phi_sgn[6]] += -a + 0.5*b + 0.5*c
    Solvation_Matrix[phi_sgn[2],phi_sgn[7]] += -0.5*a + 0.25*b - 0.5*c
    
    Solvation_Matrix[phi_sgn[3],phi_sgn[0]] += 0.5*a - b + 0.5*c
    Solvation_Matrix[phi_sgn[3],phi_sgn[1]] += 0.25*a - 0.5*b - 0.5*c
    Solvation_Matrix[phi_sgn[3],phi_sgn[2]] += 0.5*a + 0.5*b - c
    Solvation_Matrix[phi_sgn[3],phi_sgn[3]] += a + b + c
    Solvation_Matrix[phi_sgn[3],phi_sgn[4]] += -0.5*a - 0.5*b + 0.25*c
    Solvation_Matrix[phi_sgn[3],phi_sgn[5]] += -0.25*a - 0.25*b - 0.25*c
    Solvation_Matrix[phi_sgn[3],phi_sgn[6]] += -0.5*a + 0.25*b - 0.5*c
    Solvation_Matrix[phi_sgn[3],phi_sgn[7]] += -a + 0.5*b + 0.5*c
    
    Solvation_Matrix[phi_sgn[4],phi_sgn[0]] += -a + 0.5*b + 0.5*c
    Solvation_Matrix[phi_sgn[4],phi_sgn[1]] += -0.5*a + 0.25*b - 0.5*c
    Solvation_Matrix[phi_sgn[4],phi_sgn[2]] += -0.25*a - 0.25*b - 0.25*c
    Solvation_Matrix[phi_sgn[4],phi_sgn[3]] += -0.5*a - 0.5*b + 0.25*c
    Solvation_Matrix[phi_sgn[4],phi_sgn[4]] += a + b + c
    Solvation_Matrix[phi_sgn[4],phi_sgn[5]] += 0.5*a + 0.5*b - c
    Solvation_Matrix[phi_sgn[4],phi_sgn[6]] += 0.25*a - 0.5*b - 0.5*c
    Solvation_Matrix[phi_sgn[4],phi_sgn[7]] += 0.5*a - b + 0.5*c
    
    Solvation_Matrix[phi_sgn[5],phi_sgn[0]] += -0.5*a + 0.25*b - 0.5*c
    Solvation_Matrix[phi_sgn[5],phi_sgn[1]] += -a + 0.5*b + 0.5*c
    Solvation_Matrix[phi_sgn[5],phi_sgn[2]] += -0.5*a - 0.5*b + 0.25*c
    Solvation_Matrix[phi_sgn[5],phi_sgn[3]] += -0.25*a - 0.25*b - 0.25*c
    Solvation_Matrix[phi_sgn[5],phi_sgn[4]] += 0.5*a + 0.5*b - c
    Solvation_Matrix[phi_sgn[5],phi_sgn[5]] += a + b + c
    Solvation_Matrix[phi_sgn[5],phi_sgn[6]] += 0.5*a - b + 0.5*c
    Solvation_Matrix[phi_sgn[5],phi_sgn[7]] += 0.25*a - 0.5*b - 0.5*c
    
    Solvation_Matrix[phi_sgn[6],phi_sgn[0]] += -0.25*a - 0.25*b - 0.25*c
    Solvation_Matrix[phi_sgn[6],phi_sgn[1]] += -0.5*a - 0.5*b + 0.25*c
    Solvation_Matrix[phi_sgn[6],phi_sgn[2]] += -a + 0.5*b + 0.5*c
    Solvation_Matrix[phi_sgn[6],phi_sgn[3]] += -0.5*a + 0.25*b - 0.5*c
    Solvation_Matrix[phi_sgn[6],phi_sgn[4]] += 0.25*a - 0.5*b - 0.5*c
    Solvation_Matrix[phi_sgn[6],phi_sgn[5]] += 0.5*a - b + 0.5*c
    Solvation_Matrix[phi_sgn[6],phi_sgn[6]] += a + b + c
    Solvation_Matrix[phi_sgn[6],phi_sgn[7]] += 0.5*a + 0.5*b - c
    
    Solvation_Matrix[phi_sgn[7],phi_sgn[0]] += -0.5*a - 0.5*b + 0.25*c
    Solvation_Matrix[phi_sgn[7],phi_sgn[1]] += -0.25*a - 0.25*b - 0.25*c
    Solvation_Matrix[phi_sgn[7],phi_sgn[2]] += -0.5*a + 0.25*b - 0.5*c
    Solvation_Matrix[phi_sgn[7],phi_sgn[3]] += -a + 0.5*b + 0.5*c
    Solvation_Matrix[phi_sgn[7],phi_sgn[4]] += 0.5*a - b + 0.5*c
    Solvation_Matrix[phi_sgn[7],phi_sgn[5]] += 0.25*a - 0.5*b - 0.5*c
    Solvation_Matrix[phi_sgn[7],phi_sgn[6]] += 0.5*a + 0.5*b - c
    Solvation_Matrix[phi_sgn[7],phi_sgn[7]] += a + b + c

# 由电势信息构造 A, b
for i in range(NX):
    waiting(i,NX)
    for j in range(NY):
        for k in range(NZ):
            if cubes[ijk_to_sgn(i,j,k)].if_calc == True:
                if cubes[ijk_to_sgn(i,j,k)].if_tetra == True:
                    from_tetra_generate_deviation(i,j,k)
                else:
                    from_cube_generate_deviation(i,j,k)
                    
CSR_Matrix = Solvation_Matrix.tocsr()

读出矩阵

In [ ]:
f_csr = open(r"./FGM_info/Potential_Matrix.txt", 'w')
f_csr.write(str(CSR_Matrix.shape[0]) + ' ')
f_csr.write(str(len(CSR_Matrix.data)) + ' ')
f_csr.write(str(NX) + ' ')
f_csr.write(str(NY) + ' ')
f_csr.write(str(NZ) + ' ')
f_csr.write(str(box_size[0]) + ' ')
f_csr.write(str(box_size[1]) + ' ')
f_csr.write(str(box_size[2]) + ' ')
f_csr.write(str(first_cut_sgn) + '\n')
    
for i in range(len(CSR_Matrix.indices)):
    f_csr.write(str(int(CSR_Matrix.indices[i])))
    f_csr.write(' ')
f_csr.write('\n') 
for i in range(len(CSR_Matrix.indptr)):
    f_csr.write(str(int(CSR_Matrix.indptr[i])))
    f_csr.write(' ')
f_csr.write('\n')
for i in range(len(CSR_Matrix.data)):
    f_csr.write(str(CSR_Matrix.data[i]))
    f_csr.write(' ')
f_csr.write('\n')
for i in range(len(Solvation_Const)):
    f_csr.write(str(Solvation_Const[i]))
    f_csr.write(' ')
f_csr.write('\n')
    
for i in range(len(Phi)):
    f_csr.write(str(Phi[i]))
    f_csr.write(' ')
f_csr.write('\n')

for i in range(NX*NY*NZ):
     f_csr.write(str(ortho_need_convert_list[i]) + ' ')
f_csr.write('\n')
for i in range(first_cut_sgn):
     f_csr.write(str(calc_need_convert_list[i]) + ' ')
f_csr.write('\n')

f_csr.close()